In [ ]:
#Run Once
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
#Run Once
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-04-12 21:51:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.174.24.146, 3.226.107.193, 54.235.108.207, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.174.24.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  14.06M  13.6MB/s    in 1.0s    

2021-04-12 21:51:33 (13.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://ee660445b261.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":69,"gauge":0,"rate1":2.3654413902142882e-54,"rate5":1

In [ ]:
import pandas as pd
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import *
from pyspark.sql import functions as F
sc = spark.sparkContext
from pyspark.sql import Row

In [ ]:
Final_X_Ydataset = spark.read.csv('/content/drive/MyDrive/Colab datasets/Final/Final_X_Y_3H_dataset_v2.csv', header = True, inferSchema=True)
Final_X_Ydataset.printSchema()

root
 |-- date: string (nullable = true)
 |-- rain_OAK_PARK: double (nullable = true)
 |-- temp_OAK_PARK: double (nullable = true)
 |-- rhum_OAK_PARK: integer (nullable = true)
 |-- msl_OAK_PARK: double (nullable = true)
 |-- wdsp_OAK_PARK: integer (nullable = true)
 |-- wddir_OAK_PARK: integer (nullable = true)
 |-- rain_SHANNON_AIRPORT: double (nullable = true)
 |-- temp_SHANNON_AIRPORT: double (nullable = true)
 |-- rhum_SHANNON_AIRPORT: integer (nullable = true)
 |-- msl_SHANNON_AIRPORT: double (nullable = true)
 |-- wdsp_SHANNON_AIRPORT: integer (nullable = true)
 |-- wddir_SHANNON_AIRPORT: integer (nullable = true)
 |-- rain_ATHENRY: double (nullable = true)
 |-- temp_ATHENRY: double (nullable = true)
 |-- rhum_ATHENRY: integer (nullable = true)
 |-- msl_ATHENRY: double (nullable = true)
 |-- wdsp_ATHENRY: integer (nullable = true)
 |-- wddir_ATHENRY: integer (nullable = true)
 |-- rain_BELMULLET: double (nullable = true)
 |-- temp_BELMULLET: double (nullable = true)
 |-- rhum_BE

In [ ]:
Final_X_Ydataset.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
date,87670,None,None,01/01/2011 00:00,31/12/2020 21:00
rain_OAK_PARK,87670,0.09659974905897614,0.4225465948892593,0.0,17.8
temp_OAK_PARK,87670,10.125015398654057,5.177328962183774,-5.6,29.5
rhum_OAK_PARK,87670,82.761161172579,12.584318702595978,20,100
...,...,...,...,...,...
wddir_Gurteen,87670,200.84772442112467,79.66354074464343,0,360
rain_sligo_Y3H,87670,0.1453005589141267,0.4733685930539768,0.0,13.2
temp_sligo_Y3H,87670,9.464827192882412,4.913974583831496,-8.4,29.4
rhum_sligo_Y3H,87670,83.86566670468804,11.95277761570073,0,100


In [ ]:
## See if we have missing values
from pyspark.sql.functions import isnull, when, count, col

Final_X_Ydataset.select([count(when(isnull(c), c)).alias(c) for c in Final_X_Ydataset.columns]).show()

+----+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+------------+------------+------------+-----------+------------+-------------+--------------+--------------+--------------+-------------+--------------+---------------+-------------+-------------+-------------+------------+-------------+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+-----------+-----------+-----------+----------+-----------+------------+--------------+--------------+--------------+-------------+--------------+---------------+------------------+------------------+--

In [ ]:
Final_X_Ydataset = Final_X_Ydataset.na.fill(value=0)

In [ ]:
Final_X_Ydataset.select([count(when(isnull(c), c)).alias(c) for c in Final_X_Ydataset.columns]).show()

+----+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+------------+------------+------------+-----------+------------+-------------+--------------+--------------+--------------+-------------+--------------+---------------+-------------+-------------+-------------+------------+-------------+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+-----------+-----------+-----------+----------+-----------+------------+--------------+--------------+--------------+-------------+--------------+---------------+------------------+------------------+--

In [ ]:
Final_X_Ydataset.columns

['date',
 'rain_OAK_PARK',
 'temp_OAK_PARK',
 'rhum_OAK_PARK',
 'msl_OAK_PARK',
 'wdsp_OAK_PARK',
 'wddir_OAK_PARK',
 'rain_SHANNON_AIRPORT',
 'temp_SHANNON_AIRPORT',
 'rhum_SHANNON_AIRPORT',
 'msl_SHANNON_AIRPORT',
 'wdsp_SHANNON_AIRPORT',
 'wddir_SHANNON_AIRPORT',
 'rain_ATHENRY',
 'temp_ATHENRY',
 'rhum_ATHENRY',
 'msl_ATHENRY',
 'wdsp_ATHENRY',
 'wddir_ATHENRY',
 'rain_BELMULLET',
 'temp_BELMULLET',
 'rhum_BELMULLET',
 'msl_BELMULLET',
 'wdsp_BELMULLET',
 'wddir_BELMULLET',
 'rain_CASEMENT',
 'temp_CASEMENT',
 'rhum_CASEMENT',
 'msl_CASEMENT',
 'wdsp_CASEMENT',
 'wddir_CASEMENT',
 'rain_CLAREMORRIS',
 'temp_CLAREMORRIS',
 'rhum_CLAREMORRIS',
 'msl_CLAREMORRIS',
 'wdsp_CLAREMORRIS',
 'wddir_CLAREMORRIS',
 'rain_CORK_AIRPORT',
 'temp_CORK_AIRPORT',
 'rhum_CORK_AIRPORT',
 'msl_CORK_AIRPORT',
 'wdsp_CORK_AIRPORT',
 'wddir_CORK_AIRPORT',
 'rain_DUBLIN_AIRPORT',
 'temp_DUBLIN_AIRPORT',
 'rhum_DUBLIN_AIRPORT',
 'msl_DUBLIN_AIRPORT',
 'wdsp_DUBLIN_AIRPORT',
 'wddir_DUBLIN_AIRPORT',
 'rain_

In [ ]:
rain = Final_X_Ydataset.select( 'rain_OAK_PARK',
 'temp_OAK_PARK',
 'rhum_OAK_PARK',
 'msl_OAK_PARK',
 'wdsp_OAK_PARK',
 'wddir_OAK_PARK',
 'rain_SHANNON_AIRPORT',
 'temp_SHANNON_AIRPORT',
 'rhum_SHANNON_AIRPORT',
 'msl_SHANNON_AIRPORT',
 'wdsp_SHANNON_AIRPORT',
 'wddir_SHANNON_AIRPORT',
 'rain_DUBLIN_AIRPORT',
 'temp_DUBLIN_AIRPORT',
 'rhum_DUBLIN_AIRPORT',
 'msl_DUBLIN_AIRPORT',
 'wdsp_DUBLIN_AIRPORT',
 'wddir_DUBLIN_AIRPORT',
 'rain_MOORE_PARK',
 'temp_MOORE_PARK',
 'rhum_MOORE_PARK',
 'msl_MOORE_PARK',
 'wdsp_MOORE_PARK',
 'wddir_MOORE_PARK',
 'rain_Sherk_inIsland',
 'temp_Sherk_inIsland',
 'rhum_Sherk_inIsland',
 'msl_Sherk_inIsland',
 'wdsp_Sherk_inIsland',
 'wddir_Sherk_inIsland',
 'rain_MULLINGAR',
 'temp_MULLINGAR',
 'rhum_MULLINGAR',
 'msl_MULLINGAR',
 'wdsp_MULLINGAR',
 'wddir_MULLINGAR',
 'rain_MALIN_HEAD',
 'temp_MALIN_HEAD',
 'rhum_MALIN_HEAD',
 'msl_MALIN_HEAD',
 'wdsp_MALIN_HEAD',
 'wddir_MALIN_HEAD',
 'rain_JOHNSTOWN',
 'temp_JOHNSTOWN',
 'rhum_JOHNSTOWN',
 'msl_JOHNSTOWN',
 'wdsp_JOHNSTOWN',
 'wddir_JOHNSTOWN',
 'rain_MT_DILLON',
 'temp_MT_DILLON',
 'rhum_MT_DILLON',
 'msl_MT_DILLON',
 'wdsp_MT_DILLON',
 'wddir_MT_DILLON',
 'rain_FINNER',
 'temp_FINNER',
 'rhum_FINNER',
 'msl_FINNER',
 'wdsp_FINNER',
 'wddir_FINNER',
 'rain_CLAREMORRIS',
 'temp_CLAREMORRIS',
 'rhum_CLAREMORRIS',
 'msl_CLAREMORRIS',
 'wdsp_CLAREMORRIS',
 'wddir_CLAREMORRIS',
 'rain_VALENTIA_OBSERVATORY',
 'temp_VALENTIA_OBSERVATORY',
 'rhum_VALENTIA_OBSERVATORY',
 'msl_VALENTIA_OBSERVATORY',
 'wdsp_VALENTIA_OBSERVATORY',
 'wddir_VALENTIA_OBSERVATORY',
 'rain_BELMULLET',
 'temp_BELMULLET',
 'rhum_BELMULLET',
 'msl_BELMULLET',
 'wdsp_BELMULLET',
 'wddir_BELMULLET',
 'rain_CASEMENT',
 'temp_CASEMENT',
 'rhum_CASEMENT',
 'msl_CASEMENT',
 'wdsp_CASEMENT',
 'wddir_CASEMENT',
 'rain_CORK_AIRPORT',
 'temp_CORK_AIRPORT',
 'rhum_CORK_AIRPORT',
 'msl_CORK_AIRPORT',
 'wdsp_CORK_AIRPORT',
 'wddir_CORK_AIRPORT',
 'rain_KNOCK_AIRPORT',
 'temp_KNOCK_AIRPORT',
 'rhum_KNOCK_AIRPORT',
 'msl_KNOCK_AIRPORT',
 'wdsp_KNOCK_AIRPORT',
 'wddir_KNOCK_AIRPORT',
 'rain_Newport',
 'temp_Newport',
 'rhum_Newport',
 'msl_Newport',
 'wdsp_Newport',
 'wddir_Newport',
 'rain_RochesPoint',
 'temp_RochesPoint',
 'rhum_RochesPoint',
 'msl_RochesPoint',
 'wdsp_RochesPoint',
 'wddir_RochesPoint',
 'rain_ATHENRY',
 'temp_ATHENRY',
 'rhum_ATHENRY',
 'msl_ATHENRY',
 'wdsp_ATHENRY',
 'wddir_ATHENRY',
 'rain_Markree',
 'temp_Markree',
 'rhum_Markree',
 'msl_Markree',
 'wdsp_Markree',
 'wddir_Markree',
 'rain_Dunsany',
 'temp_Dunsany',
 'rhum_Dunsany',
 'msl_Dunsany',
 'wdsp_Dunsany',
 'wddir_Dunsany',
 'rain_BALLYHAISE',
 'temp_BALLYHAISE',
 'rhum_BALLYHAISE',
 'msl_BALLYHAISE',
 'wdsp_BALLYHAISE',
 'wddir_BALLYHAISE',
 'rain_Gurteen',
 'temp_Gurteen',
 'rhum_Gurteen',
 'msl_Gurteen',
 'wdsp_Gurteen',
 'wddir_Gurteen',
 'rain_sligo_Y3H')

In [ ]:
rain.show(5)

+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+---------------+---------------+---------------+--------------+---------------+----------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------+--------------+--------------+-------------+--------------+---------------+---------------+---------------+---------------+--------------+---------------+----------------+--------------+--------------+--------------+-------------+--------------+---------------+--------------+--------------+--------------+-------------+--------------+---------------+-----------+-----------+-----------+----------+-----------+------------+---------------

In [ ]:
import six
for i in rain.columns:
    if not( isinstance(rain.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to Pridicted Rain for ", i, rain.stat.corr('rain_sligo_Y3H',i))

Correlation to Pridicted Rain for  rain_OAK_PARK 0.12370473493142935
Correlation to Pridicted Rain for  temp_OAK_PARK 0.010682836105653135
Correlation to Pridicted Rain for  rhum_OAK_PARK 0.06184296773750019
Correlation to Pridicted Rain for  msl_OAK_PARK -0.058485153092974
Correlation to Pridicted Rain for  wdsp_OAK_PARK 0.20293327135999462
Correlation to Pridicted Rain for  wddir_OAK_PARK -0.041029720960848046
Correlation to Pridicted Rain for  rain_SHANNON_AIRPORT 0.21209919327169038
Correlation to Pridicted Rain for  temp_SHANNON_AIRPORT 0.010798685395706392
Correlation to Pridicted Rain for  rhum_SHANNON_AIRPORT 0.089572635758251
Correlation to Pridicted Rain for  msl_SHANNON_AIRPORT -0.20079306098717295
Correlation to Pridicted Rain for  wdsp_SHANNON_AIRPORT 0.1988379068550303
Correlation to Pridicted Rain for  wddir_SHANNON_AIRPORT 0.013421718190497336
Correlation to Pridicted Rain for  rain_DUBLIN_AIRPORT 0.09444915373985362
Correlation to Pridicted Rain for  temp_DUBLIN_AIRPOR

In [ ]:
featurename =[]
corList= []

for i in rain.columns:
    if not( isinstance(rain.select(i).take(1)[0][0], six.string_types)):
      fbuild = []
      cbuild = []
      my_cor = rain.stat.corr('rain_sligo_Y3H', i)
      
      featurename.append(i)
      corList.append(my_cor )

      print( "Correlation to Pridicted Rain for ", i, my_cor)

corrtable = [featurename, corList]

Correlation to Pridicted Rain for  rain_OAK_PARK 0.12370473493142935
Correlation to Pridicted Rain for  temp_OAK_PARK 0.010682836105653135
Correlation to Pridicted Rain for  rhum_OAK_PARK 0.06184296773750019
Correlation to Pridicted Rain for  msl_OAK_PARK -0.058485153092974
Correlation to Pridicted Rain for  wdsp_OAK_PARK 0.20293327135999462
Correlation to Pridicted Rain for  wddir_OAK_PARK -0.041029720960848046
Correlation to Pridicted Rain for  rain_SHANNON_AIRPORT 0.21209919327169038
Correlation to Pridicted Rain for  temp_SHANNON_AIRPORT 0.010798685395706392
Correlation to Pridicted Rain for  rhum_SHANNON_AIRPORT 0.089572635758251
Correlation to Pridicted Rain for  msl_SHANNON_AIRPORT -0.20079306098717295
Correlation to Pridicted Rain for  wdsp_SHANNON_AIRPORT 0.1988379068550303
Correlation to Pridicted Rain for  wddir_SHANNON_AIRPORT 0.013421718190497336
Correlation to Pridicted Rain for  rain_DUBLIN_AIRPORT 0.09444915373985362
Correlation to Pridicted Rain for  temp_DUBLIN_AIRPOR

In [ ]:
corrtable

[['rain_OAK_PARK',
  'temp_OAK_PARK',
  'rhum_OAK_PARK',
  'msl_OAK_PARK',
  'wdsp_OAK_PARK',
  'wddir_OAK_PARK',
  'rain_SHANNON_AIRPORT',
  'temp_SHANNON_AIRPORT',
  'rhum_SHANNON_AIRPORT',
  'msl_SHANNON_AIRPORT',
  'wdsp_SHANNON_AIRPORT',
  'wddir_SHANNON_AIRPORT',
  'rain_DUBLIN_AIRPORT',
  'temp_DUBLIN_AIRPORT',
  'rhum_DUBLIN_AIRPORT',
  'msl_DUBLIN_AIRPORT',
  'wdsp_DUBLIN_AIRPORT',
  'wddir_DUBLIN_AIRPORT',
  'rain_MOORE_PARK',
  'temp_MOORE_PARK',
  'rhum_MOORE_PARK',
  'msl_MOORE_PARK',
  'wdsp_MOORE_PARK',
  'wddir_MOORE_PARK',
  'rain_Sherk_inIsland',
  'temp_Sherk_inIsland',
  'rhum_Sherk_inIsland',
  'msl_Sherk_inIsland',
  'wdsp_Sherk_inIsland',
  'wddir_Sherk_inIsland',
  'rain_MULLINGAR',
  'temp_MULLINGAR',
  'rhum_MULLINGAR',
  'msl_MULLINGAR',
  'wdsp_MULLINGAR',
  'wddir_MULLINGAR',
  'rain_MALIN_HEAD',
  'temp_MALIN_HEAD',
  'rhum_MALIN_HEAD',
  'msl_MALIN_HEAD',
  'wdsp_MALIN_HEAD',
  'wddir_MALIN_HEAD',
  'rain_JOHNSTOWN',
  'temp_JOHNSTOWN',
  'rhum_JOHNSTOWN'

In [ ]:



R = Row("Name", "Score")
myscores = sc.parallelize([R(*r) for r in zip(*corrtable)]).toDF()

In [ ]:
myscores.show(5)

+-------------+--------------------+
|         Name|               Score|
+-------------+--------------------+
|rain_OAK_PARK| 0.12370473493142935|
|temp_OAK_PARK|0.010682836105653135|
|rhum_OAK_PARK| 0.06184296773750019|
| msl_OAK_PARK|  -0.058485153092974|
|wdsp_OAK_PARK| 0.20293327135999462|
+-------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col

myscores.sort(col("Score").asc()).show(50)
myscores.sort(col("Score").desc()).show(50)

+--------------------+--------------------+
|                Name|               Score|
+--------------------+--------------------+
|       msl_BELMULLET|-0.22278804878990283|
|   msl_KNOCK_AIRPORT| -0.2156385589347878|
|     msl_CLAREMORRIS|-0.21454876807970594|
|      msl_MALIN_HEAD|-0.21113072085208925|
| msl_SHANNON_AIRPORT|-0.20079306098717295|
|        msl_CASEMENT| -0.1889570293713148|
|  msl_DUBLIN_AIRPORT|-0.18714156638099486|
|      msl_MOORE_PARK|-0.18400883055429562|
|    msl_CORK_AIRPORT|-0.18172827956606888|
|       msl_MULLINGAR| -0.1551362245069381|
|     msl_RochesPoint|-0.14103288574186232|
|         msl_Dunsany|-0.11230740858132883|
|         msl_Gurteen|-0.10900797383374226|
|      msl_BALLYHAISE|-0.09152809885687253|
|msl_VALENTIA_OBSE...|-0.07735886147711185|
|  msl_Sherk_inIsland|-0.07518651420671348|
|         msl_ATHENRY|-0.07313941715380473|
|         msl_Markree|-0.06881503713229131|
|        msl_OAK_PARK|  -0.058485153092974|
|       msl_MT_DILLON|-0.0571985

very interesting!

rain features seem to be be the most positively correlated with Pr-rain along with windspeed


and 

msl (Mean Sea-level Pressure) features seems to be most negativily correleated with Pr-rain, then wind-direction and temperature. 

ok time to maker the vector


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|features                                                                                                                                                                                                                                                                                      

In [2]:
vector_temp_df.show(10, truncate=True)

NameError: ignored

In [ ]:
splits = vector_rain_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='rain_sligo_Y3H', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 0.1451676014086395


In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.470925
r2: -0.000000


In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","rain_sligo_Y3H","features").show(20)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="rain_sligo_Y3H",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+--------------+--------------------+
|        prediction|rain_sligo_Y3H|            features|
+------------------+--------------+--------------------+
|0.1451676014086395|           0.2|[0.0,-5.6,93.0,10...|
|0.1451676014086395|           0.0|[0.0,-5.6,100.0,1...|
|0.1451676014086395|           0.0|[0.0,-5.1,96.0,10...|
|0.1451676014086395|           0.0|[0.0,-5.0,100.0,1...|
|0.1451676014086395|           0.0|[0.0,-4.9,94.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.9,100.0,1...|
|0.1451676014086395|           0.2|[0.0,-4.7,94.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.2,90.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.2,93.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.2,97.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.2,97.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.2,98.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.1,95.0,10...|
|0.1451676014086395|           0.0|[0.0,-4.0,56.0,10...|
|0.1451676014086395|           

Ahhhh!

I think the dataset is extreme biased to the 0 mm of rain. 

I need to investigate this

In [ ]:
Final_X_Ydataset.toPandas()['rain_sligo_Y3H'].unique()

array([0, 1, 2, 3, 4, 6, 5, 7, 9], dtype=int32)

In [ ]:
from pyspark.sql.functions import countDistinct
Final_X_Ydataset.select(countDistinct("rain_sligo_Y3H").show()

AnalysisException: ignored

In [ ]:
# get the dimensions of the data
(Final_X_Ydataset.count() , len(Final_X_Ydataset.columns))

(87670, 143)

In [ ]:

Final_X_Ydataset.groupBy('rain_sligo_Y3H').count().show()

+--------------+-----+
|rain_sligo_Y3H|count|
+--------------+-----+
|           0.0|86915|
|           7.0|    3|
|           1.0|  530|
|           4.0|   14|
|           3.0|   46|
|           2.0|  150|
|          0.01|    1|
|           6.0|    4|
|           5.0|    6|
|           9.0|    1|
+--------------+-----+



In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'rain_sligo_Y3H')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="rain_sligo_Y3H", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.436564


In [ ]:
dt_model.featureImportances

SparseVector(138, {6: 0.0109, 10: 0.0078, 18: 0.0113, 26: 0.0035, 39: 0.0367, 54: 0.0024, 60: 0.2506, 66: 0.0584, 70: 0.0079, 72: 0.4507, 77: 0.0057, 89: 0.0145, 90: 0.0218, 92: 0.0072, 95: 0.0066, 96: 0.0289, 103: 0.0212, 108: 0.0201, 114: 0.0294, 125: 0.0044})

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'rain_sligo_Y3H', maxIter=120)
gbt_model = gbt.fit(test_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'rain_sligo_Y3H', 'features').show(5)

+--------------------+--------------+--------------------+
|          prediction|rain_sligo_Y3H|            features|
+--------------------+--------------+--------------------+
|  0.4250742093728897|           0.2|[0.0,-5.6,93.0,10...|
|-0.00379253611048...|           0.0|[0.0,-5.6,100.0,1...|
|3.531769138022573...|           0.0|[0.0,-5.1,96.0,10...|
|-0.01057471320107812|           0.0|[0.0,-5.0,100.0,1...|
|0.018341616211717836|           0.0|[0.0,-4.9,94.0,10...|
+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
dt_evaluator = RegressionEvaluator(
    labelCol="rain_sligo_Y3H", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.23392


In [ ]:
dt_evaluator = RegressionEvaluator(
    labelCol="rain_sligo_Y3H", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator.evaluate(gbt_predictions)
print("R2 on test data = %g" % r2)

R2 on test data = 0.761515


In [ ]:
gbt_predictions.select('prediction', 'rain_sligo_Y3H', 'features').show(50)

+--------------------+--------------+--------------------+
|          prediction|rain_sligo_Y3H|            features|
+--------------------+--------------+--------------------+
|  0.4250742093728897|           0.2|[0.0,-5.6,93.0,10...|
|-0.00379253611048...|           0.0|[0.0,-5.6,100.0,1...|
|3.531769138022573...|           0.0|[0.0,-5.1,96.0,10...|
|-0.01057471320107812|           0.0|[0.0,-5.0,100.0,1...|
|0.018341616211717836|           0.0|[0.0,-4.9,94.0,10...|
|-0.01229609399606...|           0.0|[0.0,-4.9,100.0,1...|
| 0.36676960806150666|           0.2|[0.0,-4.7,94.0,10...|
|9.331119899874128E-4|           0.0|[0.0,-4.2,90.0,10...|
|0.007732299359661738|           0.0|[0.0,-4.2,93.0,10...|
|0.005253627464402522|           0.0|[0.0,-4.2,97.0,10...|
|-0.01128989323489...|           0.0|[0.0,-4.2,97.0,10...|
|0.006821741864024314|           0.0|[0.0,-4.2,98.0,10...|
|0.002282155360590...|           0.0|[0.0,-4.1,95.0,10...|
|0.023091506039230957|           0.0|[0.0,-4.0,56.0,10..